In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import pandas as pd
import cv2

from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#目前问题：尝试LRN的实现；以及目前的效果都很不好，需要考虑使用数据增强，如果仍然不行的话就要换网络，试试VGG的效果
#另一个问题是使用binary_crossentropy二分类交叉熵会报错，目前只能用多分类的，不知道这个问题解决能不能提高成功率
#了解one-hot编码，应用随机增强
#原来灰度图不是只有0和1啊卧槽，我还需要进行标准化，这是个关键问题，和数据增强一起

KeyboardInterrupt: 

In [ ]:
alldata= np.load(file="alldata_3D.npy")
alllable=pd.read_csv('label_binary.csv')
label=[]
for i in range(3000):
    data = alllable['label'][i]
    if data == 'no_tumor':
        #label.append(data)
        label.append(0)
    else:
        #label.append('meningioma_tumor')
        label.append(1)
label = np.array(label)
#按照6：2：2的结构来分成训练集，测试集和验证集
train_images,test_images,train_labels,test_labels=train_test_split(alldata,label,test_size=0.2,random_state=0)
train_images,val_images,train_labels,val_labels=train_test_split(train_images,train_labels,test_size=0.25,random_state=0)

In [ ]:
class_names1 = ['tumor','no_tumor']

In [ ]:
#train_image_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,rotation_range=40,width_shift_range=0.2,height_shift_range=0.2,zoom_range=0.2,
#horizontal_flip=True,fill_mode='nearest')

In [ ]:
#用多层感知机的话数据格式需要改一下，之后在处理
'''model = keras.models.Sequential()
model.add(keras.layers.Dense(64, input_dim=20, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.summary()'''

In [ ]:
model = keras.Sequential()
covn_base=keras.applications.xception.Xception(include_top=False, weights=None,
                                    input_tensor=None, input_shape=(224, 224, 3),
                                    pooling=None, classes=1000)
model.add(covn_base)
model.add(keras.layers.Dense(2048, input_shape=(7,7,2048),activation='relu'))
model.add(keras.layers.Dense(2,activation="softmax"))
model.summary()

In [ ]:
train_images.shape

In [ ]:
#损失函数应该改一下，这个是针对多分类的，二分类的不一样binary_crossentropy，优化器可能也要改
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])
#使用二分类交叉熵会报错，这是个大问题？需要解决
#epoch 迭代次数
#model.fit 方法返回一个 History 回调，它具有包含连续误差的列表和其他度量的 history 属性。
history =model.fit(train_images, train_labels, batch_size=32, epochs=15,validation_data=(test_images, test_labels))

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
plt.show()

test_loss, test_acc = model.evaluate(val_images, val_labels, verbose=2)
print(test_acc)

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='loss')
plt.xlabel('Epoch')
plt.ylabel('Lose')
plt.ylim([0, 2])
plt.legend(loc='lower right')
plt.show()
print(test_loss)